In [61]:
import sys
import os
import gc
import time
import random
import re
from tqdm import tqdm 
import json
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM, BatchEncoding
import matplotlib.pyplot as plt
from math import ceil
from sentence_transformers import SentenceTransformer
from datetime import datetime
from math import sqrt

In [2]:
sys.path.append("/coc/pskynet6/dhe83/mice/src")

In [3]:
import config
import utils
from prompts import *

In [4]:
from datasets import load_dataset

dataset = load_dataset("gsm8k","main")

Found cached dataset gsm8k (/nethome/dhe83/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model_addr = "decapoda-research/llama-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_addr, use_fast=False, padding_side='left')
model = LlamaForCausalLM.from_pretrained(
    model_addr,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map='auto'
)
# unwind broken decapoda-research config
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)
print("done!")

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/libcudart.so'), PosixPath('/srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

done!


In [6]:
torch.cuda.mem_get_info()

(39487602688, 47850782720)

In [7]:
def get_label(ex):
    return ex['answer'].split("####")[-1].strip()

def format_example(ex, in_context=False):
    prompt = f"Q: {ex['question']}\n\nA:"
    
    if in_context:
        prompt = " ".join([prompt, ex['answer']])
        prompt = re.sub("(<<).*?(>>)", "", prompt)
        prompt = re.sub("\\n(#+).*", f" The answer is {get_label(ex)}.", prompt)
    return prompt

def extract(output, in_context=False):
    answer = output.lower()
    answer = re.split("the answer is", answer)
    if len(answer) > 1:
        answer = answer[1] 
    else:
        return "" 
    answer = re.findall("\d+", answer)
    
    if len(answer) > 0:
        return answer[0] 
    else:
        return "" 

def format_few_shot(demonstrations, test):
    return "\n\n".join([*[format_example(ex, True) for ex in demonstrations], format_example(test)])

In [8]:
train_ids = [3366, 3561, 3965, 7261, 3169, 4861, 1382, 1743]
test_ids = [1289, 417, 828, 1211, 1134, 1139, 793, 869, 952, 506, 287, 1147, 84, 926, 941, 661, 524, 596, 581, 706, 614, 1307, 363, 33, 227, 910, 1020, 124, 858, 224, 855, 1188, 834, 757, 1295, 167, 628, 131, 213, 995, 1150, 839, 290, 838, 837, 141, 947, 1201, 845, 544, 717, 835, 759, 394, 736, 1104, 1263, 1084, 450, 114, 1283, 593, 772, 316, 1301, 1255, 203, 1069, 754, 511, 360, 595, 1222, 553, 785, 25, 774, 1059, 214, 262, 310, 275, 880, 903, 557, 121, 1203, 297, 968, 954, 1029, 667, 207, 1078, 728, 380, 1177, 970, 1017, 1308]

In [9]:
train = {x: dataset['train'][x] for x in train_ids}
test = {x:dataset['test'][x] for x in test_ids}

In [48]:
gen_len=150

In [31]:
available_bytes = torch.cuda.mem_get_info()[0]

In [70]:
def bytes_per_token(model):
    return (2 * 2
                * model.config.num_hidden_layers
                * model.config.num_attention_heads
                * sqrt(model.config.hidden_size / 4))

In [71]:
per_token = bytes_per_token(model)

In [72]:
2 * (2**20)

2097152

In [73]:
per_token

642119.0391321534

In [58]:
model.config

LlamaConfig {
  "_name_or_path": "decapoda-research/llama-7b-hf",
  "architectures": [
    "LLaMAForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "quantization_config": {
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.29.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [55]:
per_token

16777216

In [56]:
2 * 2**20

2097152

In [50]:
model.config

LlamaConfig {
  "_name_or_path": "decapoda-research/llama-7b-hf",
  "architectures": [
    "LLaMAForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": 0,
  "quantization_config": {
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.29.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [25]:
def maximum_batch_size(available_bytes, max_tokens):
    per_token = 2 * 2**20
    return available_bytes // (per_token * max_tokens)

def max_token_len(prompts, gen_len=0):
    return max([len(tokenizer(p).input_ids) for p in prompts]) + gen_len

In [44]:
def batch_inference(prompts):
    output_tokens = torch.empty(0, dtype=torch.int64).to('cuda:0')
    
    batch_size = maximum_batch_size(available_bytes, max_token_len(prompts, gen_len))
    num_batches = ceil(len(prompts) / batch_size)

    for batch in tqdm(range(num_batches)):
        start = batch * batch_size
        end = min((batch + 1) * batch_size, len(prompts))

        # tokenize by batch to mitigate effect of long outliers
        tokens = tokenizer(prompts[start:end], padding=True, return_tensors="pt").to('cuda:0')
        attention_mask = tokens.attention_mask

        with torch.no_grad():
            outputs = model.generate(
                input_ids=tokens.input_ids,
                attention_mask=attention_mask,
                max_new_tokens=gen_len,
                temperature=0.5,
                do_sample=True,
                top_k=40,
                return_dict_in_generate=True,
                output_scores=True,
            )
        output_tokens = torch.cat((output_tokens, outputs.sequences[:, -gen_len:]))
    output_text = tokenizer.batch_decode(output_tokens)
    return output_text    

In [45]:
prompt_map = {idx: format_few_shot(train.values(), ex) for idx, ex in test.items()}

In [49]:
outputs = batch_inference(list(prompt_map.values()))

  0%|                                                                                                                                                 | 0/10 [00:00<?, ?it/s]/srv/nlprx-lab/share6/dhe83/miniconda3/envs/mice/lib/python3.9/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
 20%|███████████████████████████▍                                                                                                             | 2/10 [01:15<05:01, 37.67s/it]


KeyboardInterrupt: 

In [66]:
results = {}
for idx, res in zip(prompt_map.keys(), outputs):
    answer = extract(res)
    results[idx] = {"idx": idx,
                    "prompt": prompt_map[idx], 
                    "generated_text": res, 
                    "prediction": answer,
                    "label": get_label(test[idx])}

In [67]:
acc = sum([1 for v in results.values() if v['prediction'] == v['label']])/ len(results)

In [68]:
acc

0.09

In [70]:
time = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
utils.write_jsonl(results.values(), f"gsm8k_cot_7b_{time}.jsonl")